# TO DO: CARGAR LOS OUTPUTS DE LOS DOS MODELOS

In [1]:
import pandas as pd

cnn_probs_train = pd.read_csv('/kaggle/input/cnn-results/predicts_vs_real_TRAIN.csv', index_col=0)
cnn_probs_test = pd.read_csv('/kaggle/input/cnn-results/predicts_vs_real_TEST.csv', index_col=0)
cnn_columns = ['cnn_prob_0', 'cnn_prob_1', 'cnn_prob_2', 'cnn_prob_3', 'cnn_prob_4', 'cnn_prob_5', 'tag']

In [2]:
forest_probs_train = pd.read_csv('/kaggle/input/forest-results/predicts_vs_real_TRAIN.csv', index_col=0)
print('Train checker:', all(forest_probs_train.tag.values == cnn_probs_train.tag.values))
forest_probs_train.drop(columns = 'tag', inplace=True)

forest_probs_test = pd.read_csv('/kaggle/input/forest-results/predicts_vs_real_TEST.csv', index_col=0)
print('Test checker:', all(forest_probs_test.tag.values == cnn_probs_test.tag.values))
forest_probs_test.drop(columns = 'tag', inplace=True)
forest_columns = ['forest_prob_0', 'forest_prob_1', 'forest_prob_2', 'forest_prob_3', 'forest_prob_4', 'forest_prob_5']

Train checker: True
Test checker: True


# TO DO: CREAR DOS CONJUNTOS TRAIN Y TEST CON SUS X E Y

In [3]:
cnn_probs_train.columns = cnn_probs_test.columns = cnn_columns
forest_probs_train.columns = forest_probs_test.columns = forest_columns

x_train = pd.concat([forest_probs_train, cnn_probs_train], axis=1)
y_train = x_train.tag
x_train.drop(columns='tag', inplace=True)

x_test = pd.concat([forest_probs_test, cnn_probs_test], axis=1)
y_test = x_test.tag
x_test.drop(columns='tag', inplace=True)

# TO DO: ENTRENAR UN MODELO DE REGRESION QUE HAGA DE ENSEMBLE ENTRE LOS DOS DE PARTIDA

In [4]:
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report

lr_model = LogisticRegression(max_iter=5000)

parameters = {}
parameters['C'] = [10e-3, 10e-2, 10e-1, 1, 10, 100, 1000]
parameters['class_weight'] = [None, 'balanced']

GS = GridSearchCV(lr_model, parameters)
GS.fit(x_train, y_train)

lr_model = GS.best_estimator_

pred_probs_test = lr_model.predict_proba(x_test)
y_test_pred = np.argmax(pred_probs_test, axis=1)
report = classification_report(y_test, y_test_pred, output_dict=True)
pd.DataFrame(report).T

,precision,recall,f1-score,support
0,0.674850,0.808884,0.735813,14138.000000
1,0.813247,0.260460,0.394555,3394.000000
2,0.358523,0.313230,0.334350,11905.000000
3,0.764191,0.727322,0.745301,7441.000000
4,0.571969,0.628118,0.598730,10584.000000
5,0.630064,0.645740,0.637806,12982.000000
accuracy,0.603732,0.603732,0.603732,0.603732
macro avg,0.635474,0.563959,0.574426,60444.000000
weighted avg,0.603682,0.603732,0.593693,60444.000000


Mejoramos **~1%** con respecto a la media aritmetica de las predicciones